1. 引用我自己寫的 helper function: query_from_db可以直接從DB獲取資料>
2. Import 會用到的 library
3. 定義了python request 連線的 header

In [1]:
# Import the pre-implement db query function

import os, sys
parent_dir = os.path.dirname(os.getcwd())
#os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(parent_dir)
from db_func import query_from_db
import html
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time

headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}

##### 將資料print出來的function，方便debug。

In [2]:
def print_util(urls, processor):
    for url in urls:
        try:
            print()
            temp = processor(url[0])
            print('News Url: \n{}\n'.format(url[0]))
            #print("Title: {}".format(temp['news_title']))
            #print("News: {}".format(temp['news']))
            for key in temp.keys():
                print('{}: {}'.format(key.capitalize(), temp[key]))
            print()
            print()
        except Exception as e:
            print()
            print(e)
            print('ERROR URL:')
            print(url)
            print()

### 選出10個在DB之中有最多資料的新聞來源：
```sql
SELECT news_source,COUNT(*) 
FROM news_db.news_rss_feeds
GROUP BY news_source
ORDER BY 2 DESC
LIMIT 11;
```

|  news_source    | COUNT(*) |
|------------------|----------|
| MSN              | 14204    |
| 自由時報         | 4997     |
| 大紀元           | 4394     |
| 經濟日報         | 3589     |
| ~新浪台灣新聞中心~ | ~3323~     |
| PCHOME           | 3213     |
| ETtoday          | 2585     |
| 新頭殼要聞       | 2117     |
| Rti 中央廣播電臺 | 2010     |
| Yahoo!奇摩股市   | 1976     |
| 公視新聞 |1796|  
最後因為新浪新聞有很多url是無效的，因此沒有放入 DB

### 1. 隨機選出50個Yahoo的新聞連結：
在開發時我是確保50個新聞連結都通過，但為了怕notebook篇幅太長，我改成五個來demo
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Yahoo!奇摩股市' ORDER BY RAND()
LIMIT 50;
```

In [3]:
random_yahoo_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source LIKE '%Yahoo%' ORDER BY RAND()
LIMIT 100; """

yahoo_test_urls = query_from_db(random_yahoo_sql)

In [4]:
def yahoo_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    stop_footers = set(['全球疫情大流行', '今日最夯新聞流量前3名', '更多相關新聞', '更多 NOWnews 今日新聞 報導 ', '看更多 CTWANT 文章', '更多上報內容：', '更多 TVBS 報導', '《更多匯流新聞網CNEWS報導》', '更多社會相關新聞', '更多東森新聞報導', '原始連結', '更多鉅亨報導', '更多國際相關新聞', '***本文所提之內容僅供參考，不具投資建議，投資前應審慎評估風險，且自負盈虧***', '延伸閱讀：', '更多華視新聞報導', '更多匯流新聞網報導：', '【延伸閱讀】', '更多民視新聞報導', '更多財經相關新聞', '※【NOWnews 今日新聞】提醒您：', '更多健康2.0報導', '更多三立新聞網報導', '超人氣', '《TVBS》提醒您：', '《更多CNEWS匯流新聞網報導》', '更多新聞推薦', '更多新頭殼報導', '更多Heho健康網文章'])
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = html.unescape(title_category[0])
        if len(title_category) > 1:
            res_dict['news_category'] = html.unescape(title_category[1])
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = html.unescape(keywords_tag['content'])

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = html.unescape(description_tag['content'])

    time_tag = soup.find('time')
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")
            db_time_format = '%Y-%m-%d %H:%M:%S'
            res_time = d1.strftime(db_time_format)
            res_dict['published_time'] = res_time

            d2 = d1 + datetime.timedelta(hours=8)
            d2 = d2.date()
            db_date_format = '%Y-%m-%d'
            res_date = d2.strftime(db_date_format)
            res_dict['published_date'] = res_date
        except Exception as e:
            logging.error(e)
            print(e)
            pass

    article_body_tag = soup.find('article', attrs = {'itemprop':'articleBody'})
    caas_body_tag = soup.find('div', attrs = {'class': 'caas-body'})
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for index, p in enumerate(p_tags):
                if p.get('content'):
                    if any([html.unescape(p.get_text()).startswith(stop_footer) for stop_footer in stop_footers]):
                        break
                    temp_content.append(html.unescape(p.get_text().strip()))
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif caas_body_tag:
        p_tags = caas_body_tag.find_all('p')
        a_tags = caas_body_tag.find_all('a')

        if p_tags:
            for index, p in enumerate(p_tags):
                if any([html.unescape(p.get_text()).startswith(stop_footer) for stop_footer in stop_footers]):
                    break
                temp_content.append(html.unescape(p.get_text()).strip())
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    if temp_content:    
        # Capture the description start with 公告 to content
        if title_category[0][:4] == '【公告】':
            prefix = title_category[0]
        else:
            prefix = ''
        content = prefix + '\n'.join(temp_content)#.replace('。 ', '。\n')
        res_dict['news'] = html.unescape(content)
        return res_dict
    else:
        print('Yahoo url: {} did not process properly'.format(url))
        #content_parser_1.logger.error('Yahoo url: {} did not process properly'.format(url))
        #content_parser.errors['process_empty_content_(rss_id)'].append([rss_id, url])
        return

In [6]:
print('Yahoo 新聞：')
print_util(yahoo_test_urls[:10].values, yahoo_content_processor)

Yahoo 新聞：

News Url: 
https://tw.news.yahoo.com/omicron%E7%87%92%E9%80%B2%E7%A4%BE%E5%8D%80-%E5%8C%97%E5%8C%97%E5%9F%BA%E6%A1%83%E9%86%AB%E9%99%A2%E7%A6%81%E6%AD%A2%E6%8E%A2%E7%97%85-063713556.html

News_title: Omicron燒進社區 北北基桃醫院禁止探病
News_keywords: 中央流行疫情指揮中心, 桃園市之醫院
News_description: 因應Omicron變異株的COVID-19本土病例延燒到社區，為確保醫療機構對疫情的因應及保全醫療量能，中央流行疫情指揮中心指揮官陳時中今天宣布，即日起調整醫院陪（探）病管制及醫療照護人員等醫療應變措施，北北基桃地區的醫院禁止探病。
Published_time: 2022-01-09 06:37:13
Published_date: 2022-01-09
News_related_url: ['https://tw.news.yahoo.com/%E4%BE%AF%E5%8F%8B%E5%AE%9C-%E6%96%B0%E5%8C%97180%E5%90%8D%E9%98%B2%E7%96%AB%E8%A8%88%E7%A8%8B%E8%BB%8A%E5%8F%B8%E6%A9%9F%E8%87%AA%E8%A1%8C%E7%AF%A9%E6%AA%A2-%E5%9D%87%E7%82%BA%E9%99%B0%E6%80%A7-033715842.html', 'https://tw.news.yahoo.com/%E5%BD%B1-%E5%8F%B0%E5%8D%97%E9%A4%90%E5%BB%B3%E6%81%A2%E5%BE%A9%E9%9A%94%E6%9D%BF-%E9%BB%83%E5%81%89%E5%93%B2-%E6%9C%89%E7%A2%BA%E8%A8%BA%E6%A1%88%E4%BE%8B%E6%9C%83%E8%80%83%E6%85%AE-033715721.html', 'https://tw.news.yahoo.com/%E4%BC%B015%E6%97%A5%

### 2. 隨機選出50個 MSN 的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 50;
```

In [7]:
random_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 100; """
msn_test_urls = query_from_db(random_msn_sql)

In [8]:

def msn_content_processor(url):
    res_dict = {}
    prefix = ['相關報導', '※', '＊', '更多三立新聞網報導', '延伸閱讀', '資料來源', '更多中時電子報精彩報導', '看了這篇文章的人，也']
    pattern = re.compile(r'^{}'.format('|'.join(prefix)))
    
    if url.startswith('https://www.msn.com/zh-tw/video/'):
        return
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        res_dict['news_title'] = html.unescape(title_tag.text.strip())
        
    
    category_tag = soup.find('div', attrs = {'class':'logowrapper'})
    if category_tag:
        if len(category_tag.get_text().strip().split('\n')) == 2:
            _, category = category_tag.get_text().strip().split('\n')
            res_dict['news_category'] = html.unescape(category)
            
    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = html.unescape(description_tag['content'])

    time_tag = soup.find('div', attrs = {'class': 'timeinfo-txt'})
    if time_tag:
        date_time_tag = time_tag.find('time')
        try:
            d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.000Z") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('MSN date error, url: {}'.format(url))
    
    article_body_tag = soup.find('section', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'content'})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    elif article_body_tag_2:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
        
    if len(temp_content):
        content = '\n'.join(temp_content)
        res_dict['news'] = html.unescape(content)

    if not res_dict or 'news' not in res_dict: 
        #content_parser.logger.error('MSN url: {} did not process properly'.format(url))
        #content_parser.errors['process_empty_content_(rss_id)'].append([rss_id, url])
        print('MSN url: {} did not process properly'.format(url))
        return 
    return res_dict

In [9]:
print('MSN 新聞：')
print_util([['https://www.msn.com/zh-tw/money/topstories/%E5%8F%B0%E7%A9%8D%E9%9B%BB%E6%89%BF%E8%AB%BE%E5%8A%A0%E9%80%9F%E4%BE%9B%E7%B5%A6-%E5%88%B0%E5%BA%95%E8%BB%8A%E7%94%A8%E6%99%B6%E7%89%87%E7%BC%BA%E8%B2%A8%E7%BC%BA%E4%BB%80%E9%BA%BC-%E8%AA%BF%E7%A0%94-%E9%80%993%E7%94%A2%E7%B7%9A%E6%9C%80%E7%B7%8A%E7%BC%BA/ar-BB1d8oqq?li=BBqj0iS&amp;srcref=rss%27']], msn_content_processor)

MSN 新聞：

News Url: 
https://www.msn.com/zh-tw/money/topstories/%E5%8F%B0%E7%A9%8D%E9%9B%BB%E6%89%BF%E8%AB%BE%E5%8A%A0%E9%80%9F%E4%BE%9B%E7%B5%A6-%E5%88%B0%E5%BA%95%E8%BB%8A%E7%94%A8%E6%99%B6%E7%89%87%E7%BC%BA%E8%B2%A8%E7%BC%BA%E4%BB%80%E9%BA%BC-%E8%AA%BF%E7%A0%94-%E9%80%993%E7%94%A2%E7%B7%9A%E6%9C%80%E7%B7%8A%E7%BC%BA/ar-BB1d8oqq?li=BBqj0iS&amp;srcref=rss%27

News_title: 台積電承諾加速供給！到底車用晶片缺貨缺什麼？調研：這3產線最緊缺
News_category: 財經
News_description: 經濟部針對全球車用晶片缺貨問題，邀請台積電、聯電、力積電、世界先進等國內主要代工廠代表一起商討。
Published_time: 2021-01-27 21:38:00
Published_date: 2021-01-28
News_related_url: []
News_related_url_desc: []
News: 編按（2021.1.28 13:30更新）：針對車用晶片缺貨問題，TrendForce報告指出，自2018年起車市逐步疲軟，加上2020年受到疫情嚴重衝擊，使主要模組廠的備貨動能明顯不足。然而，2021年全球汽車市場正在復甦，預估整車銷售量將自去年的7,700萬輛回升至8,400萬輛，同時汽車在自動化、智慧化和電動化發展下，對於各種半導體元件的用量將大幅上升，然先前因車市需求疲軟導致車廠備貨量偏低，長短料的現象已嚴重影響車廠稼動率與終端整車出貨。
因此，近期IC供應鏈的缺貨現象，已從消費性電子與電腦資通訊產業（Consumer & ICT），逐步蔓延到工控與車載市場（Industrial & Automotive）。過往車用半導體市場主要以垂直整合製造（IDM）或Fab-lite（即IDM企業將部分製造業務轉由協力廠商代工，是資產輕量化的一種策略）生產為主，例如恩智浦（NX

### 3. 隨機選出50個自由時報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 50;
```

In [10]:
ltn_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 5; """
ltn_test_urls = query_from_db(ltn_msn_sql)

In [11]:
ltn_sql_null = """
SELECT nrf.news_url 
FROM news_rss_feeds nrf
LEFT JOIN news_contents nc ON nrf.news_rss_feeds_id = nc.news_rss_feeds_id
WHERE nrf.rss_source = 'LTN' and nc.news_published_date IS NULL and nrf.processed_success = 1"""
ltn_test_urls = query_from_db(ltn_sql_null)

DatabaseError: Execution failed on sql '
SELECT nrf.news_url 
FROM news_rss_feeds nrf
LEFT JOIN news_contents nc ON nrf.news_rss_feeds_id = nc.news_rss_feeds_id
WHERE nrf.rss_source = 'LTN' and nc.news_published_date IS NULL and nrf.processed_success = 1': (1054, "Unknown column 'nc.news_published_date' in 'where clause'")

In [12]:
ltn_test_urls

,news_url
0,https://ent.ltn.com.tw/news/breakingnews/3794858
1,https://news.ltn.com.tw/news/politics/paper/14...
2,https://news.ltn.com.tw/news/life/breakingnews...
3,https://news.ltn.com.tw/news/world/breakingnew...
4,https://talk.ltn.com.tw/article/paper/1494799


In [13]:
def ltn_content_processor(url):
    postfix = ["""一手掌握經濟脈動
    點我訂閱自由財經Youtube頻道""", 
    """☆民眾如遇同居關係暴力情形，可撥打113保護專線，或向各地方政府家庭暴力防治中心求助☆""", 
    """☆健康新聞不漏接，按讚追蹤粉絲頁☆更多重要醫藥新聞訊息，請上自由健康網""",
    """《自由開講》是一個提供民眾對話的電子論壇，不論是對政治、經濟或社會、文化等新聞議題，有意見想表達、有話不吐不快，都歡迎你熱烈投稿。文長700字內為優，來稿請附真實姓名（必寫。有筆名請另註）、職業、聯絡電話、E-mail帳號。本報有錄取及刪修權，不付稿酬；請勿一稿多投，錄用與否將不另行通知。投稿信箱：LTNTALK@gmail.com""",
    """「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。""",
    """"""]
    res_dict = {}
    #headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
        else:
            try:
                res_dict['news_title'], res_dict['news_category'] = title_tag.string.split('|')
            except:
                pass
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag_1 = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('meta', attrs = {'name': 'pubdate'})
    time_tag_3 = soup.find('span', attrs = {'class': 'time'})
    if time_tag_1:
        try:
            d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%SZ")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%SZ")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))
    elif time_tag_3:
        try:
            d1 = datetime.datetime.strptime(time_tag_3.get_text(), "%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_3.get_text(), "%Y-%m-%d %H:%M:%S")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))
                

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag2 = soup.find('div', attrs = {'class':'text boxTitle boxText'})
    article_body_tag3 = soup.find('div', attrs = {'class':'text'})
    temp_content = []
    links = []
    links_descs = []

    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag2:
        p_tags = article_body_tag2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag2.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag3:
        p_tags = article_body_tag3.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag3.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        print('LTN {}, did not process properly'.format(url))
        #content_parser.logger.error('LTN url: {} did not process properly'.format(url))
        return
    return res_dict

In [14]:
print('自由時報')
print_util(ltn_test_urls[:10].values,ltn_content_processor)

自由時報

News Url: 
https://ent.ltn.com.tw/news/breakingnews/3794858

News_title: 米其林餐盤推薦Chou Chou 迎新主廚碰撞味蕾火花
News_category: 自由娛樂
News_fb_app_id: 140490219413038
News_fb_page: 524751764333507
News_keywords: 自由娛樂, ltn
News_description: 米其林餐盤推薦「Chou Chou」法式餐廳重啟營運，迎來新主廚Jason（右），左為經營者兼主廚林明健。（記者凃盈如攝）〔記者凃盈如／台北報導〕於2016年12月開業的Chou Chou法式料理餐廳，2018～2021年連續4年獲得米其林餐盤推薦，品牌步入第6年，宣布調整餐飲型態，推出Chou C
Published_time: 2022-01-08 23:40:00
Published_date: 2022-01-09
News_related_url: []
News_related_url_desc: []
News: 〔記者凃盈如／台北報導〕於2016年12月開業的Chou Chou法式料理餐廳，2018～2021年連續4年獲得米其林餐盤推薦，品牌步入第6年，宣布調整餐飲型態，推出Chou Chou 3.0全新菜單，有別於先前輕鬆的法式餐酒館，如今以fine dining定調，且品牌經營者兼主廚林明健（Chef Kin）攜手加拿大知名新日式料理餐廳Nupo前任主廚Jason（陳思凱），一同激盪精彩火花。
Chou Chou華麗轉型fine dining，冬季菜單選用鴨肝、帝王蟹、鮑魚等高檔食材，結合當令海鮮及蔬果香草，以法式料理工序為本，融入生魚熟成、茶碗蒸、炭燒、醃漬、炊飯等日式料理概念及技法，並巧妙穿插兩位主廚鍾愛的東南亞香料增添記憶點，呈盤上更現代、簡約，是細膩的料理風格，直接而美味，不需多加解釋就讓味蕾心領神會。
本次菜單當中，「鴨肝泡芙、金棗醬、茵陳蒿」，是讓人一試難忘的驚喜，外層煎至焦脆、內裡半熟滑順的鴨肝，夾入以港式菠蘿油呈現的泡芙，中間抹上宜蘭金棗、波爾多甜白酒、茵陳蒿慢煮而成的金棗醬，鴨肝濃口的油脂搭配酸甜馥郁的金棗醬，體驗味蕾的極致滿足。
擁有深厚法式料理烹飪實力的主廚林明健表示：

### 4. 隨機選出50個大紀元的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 50;
```

In [45]:
epoch_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 100; """
epoch_test_urls = query_from_db(epoch_test_sql)

In [48]:
def epoch_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ', 1)
        res_dict['news_title'] = title_category[0]
        
    
    category_tag = soup.find('meta', attrs = {'property':'article:section'})
    if category_tag:
        res_dict['news_category'] = str(category_tag['content'])
        
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Epoch date error {}'.format(e2))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'art-content'})
    #print(article_body_tag)
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag_2:
        p_tags = article_body_tag_2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag_2.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Epoch url: {} did not process properly'.format(url))

    return res_dict


In [49]:
print('大紀元：')
print_util(epoch_test_urls.values, epoch_content_processor)

大紀元：

News Url: 
https://www.epochtimes.com/b5/20/12/10/n12608555.htm

News_title: 歐冠16強出爐 皇馬後來居上 巴薩不敵尤文
News_category: 各地分網
News_fb_app_id: 1571885662850711
News_fb_page: 156786811025453
News_keywords: 16強,尤文,巴薩,曼聯,歐冠,皇馬,大紀元
News_description: 當地時間12月9日，2020/21賽季歐冠聯賽16強出爐。德甲和西甲球隊全都順利晉級。在「死亡之組」，皇馬主場2：0戰勝門興，兩隊攜手出線，國米和礦工出局；曼聯則在主場2：3不敵萊比錫，慘遭淘汰，大巴黎和萊比錫晉級。
Published_time: 2020-12-10 02:29:48
Published_date: 2020-12-10
News_related_url: ['https://www.epochtimes.com/b5/tag/%e6%ad%90%e5%86%a0.html', 'https://www.epochtimes.com/b5/tag/16%e5%bc%b7.html', 'https://www.epochtimes.com/b5/tag/%e7%9a%87%e9%a6%ac.html', 'https://www.epochtimes.com/b5/tag/%e7%9a%87%e9%a6%ac.html', 'https://www.epochtimes.com/b5/tag/%e5%b0%a4%e6%96%87.html', 'https://www.epochtimes.com/b5/tag/%e5%b7%b4%e8%96%a9.html', 'https://www.epochtimes.com/b5/tag/16%e5%bc%b7.html', 'https://www.epochtimes.com/b5/tag/%e5%b7%b4%e8%96%a9.html', 'https://www.epochtimes.com/b5/tag/%e6%ad%90%e5%86%a0.html', 'https://www.epochtimes.com/b5

### 5. 隨機選出50個經濟日報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 50;
```

In [15]:
udn_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 100; """
udn_test_urls = query_from_db(udn_test_sql)

In [16]:
import re
def udn_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category_lst = title_tag.string.split(' | ', 3)
        res_dict['news_title'] = title_category_lst[0]
        try:
            res_dict['news_category'] = title_category_lst[2]
        except:
            pass
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'shareBar__info--author'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.find('span').text, "%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d_temp = re.search('(\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2})', time_tag.text).group(0)
                d1 = datetime.datetime.strptime(d_temp, "%Y-%m-%d %H:%M") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                try:
                    d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%d %H:%M:%S") 
                    d2 = d1.date()
                    d1 -= datetime.timedelta(hours=8)
                    db_time_format = '%Y-%m-%d %H:%M:%S'
                    time_res = d1.strftime(db_time_format)
                    res_dict['published_time'] = time_res
                    
                    db_date_format = '%Y-%m-%d'
                    date_res = d2.strftime(db_date_format)
                    res_dict['published_date'] = date_res
                except Exception as e3:
                    print(e3)
                    content_parser.logger.info('Epoch date error {}'.format(e3))
    article_body_tag = soup.find('div', attrs = {'id':'article_body'})
    content_temp, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.get_text().strip() and p.get_text().strip() != 'facebook':
                    content_temp.append(p.get_text().strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs      
    content = '\n'.join(content_temp).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('udn url: {} did not process properly'.format(url))

    return res_dict


In [17]:
print_util(udn_test_urls.values, udn_content_processor)

### 6. 隨機選出50個新浪台灣新聞中心的新聞連結： (skip this phase, too many error urls)
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 50;
```

In [18]:
sina_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 20; """
sina_test_urls = query_from_db(sina_test_sql)

In [19]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [20]:
import requests
from itertools import cycle
import traceback
#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [21]:
import random
# This data was created by using the curl method explained above
headers_list = [
    # Firefox 77 Mac
     {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
        "Connection": "keep-alive",
        "DNT": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
        # Chrome 83 Windows 
    {
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9"
    }
]

def sina_content_processor(url):
    res_dict = {}
   # proxy = next(proxy_pool)
    r = requests.get(url, headers = random.choice(headers_list))
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-1].strip()
    
    
    # Sina did not have FB available
    #fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    #if fb_app_tag:
    #    res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']
        
    time_tag = soup.find('meta', attrs = {'name': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Sina date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'id':'article_content'})
    article_body_tag_2 = soup.find('div', attrs = {'class': 'pcont'})
    if article_body_tag:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content
    elif article_body_tag_2:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Sina url: {} did not process properly'.format(url))

    return res_dict


In [22]:
print_util(sina_test_urls.values, sina_content_processor)


News Url: 
https://news.sina.com.tw/article/20220109/40989162.html

News_title: 爽搭轎車竟暈了！2豬「狂拉猛吐」金黃稀屎噴散　噁心圖曝…網笑翻：殺豬刀秒拿出
News_category: 新浪新聞中心
News_keywords: 爽搭轎車竟暈了！2豬「狂拉猛吐」金黃稀屎噴散　噁心圖曝…網笑翻：殺豬刀秒拿出,中天新聞,國際新聞,新浪新聞中心
News_description: 記者陳冠萍／綜合報導省錢不成，有時反而要花費更多！一名男子為省下租車錢，因此開著自己的轎車拉豬回家；怎料，車子才剛開2公里，豬隻竟開始暈車，在轎車內上吐下瀉，把環境弄得一團糟。相關畫面曝光後，網友也紛...
News: 記者陳冠萍／綜合報導
省錢不成，有時反而要花費更多！一名男子為省下租車錢，因此開著自己的轎車拉豬回家；怎料，車子才剛開2公里，豬隻竟開始暈車，在轎車內上吐下瀉，把環境弄得一團糟。相關畫面曝光後，網友也紛紛開玩笑說：「殺豬刀給我拿出來！」
綜合陸媒報導，一名李姓男子1月7日要載豬回家時，為了省下租車費用，所以選擇開著自家轎車載2隻豬回家。怎料，車子才剛開了約2公里的路程，豬隻卻暈車了，不舒服的豬開始上吐下瀉，把車內環境搞得一團糟。
從曝光影片中可以看到，李男車上全是黃色嘔吐物，且噴散一地、幾乎整個後座都遭波及，看起來非常可怕。李男表示，2隻豬本來是關在後車廂的，但他們自己鑽到後座處，暈車後就吐了。據悉，這場意外還害李男花了80元（約新台幣346元）洗車。
相關消息與影片曝光後，也吸引許多網友關注，大家紛紛表示「看了不舒服，太崩潰了」、「這台車往後應該會瀰漫特殊味道」、「太氣了，殺豬刀拿出來了」、「現場表演殺豬，太氣了」、「男子崩潰吧，哈哈哈」、「殺豬刀秒拿出，笑死」。
【更多中天快點TV報導】化療60次「肺剩一半」！余苑綺抗癌指甲全變黑　親吐：深刻體會那種痛賴清德嗆顏家「金盆洗手」　PTT怒嗆人設崩壞…昔支持者傻眼：你變了！打第3劑BNT突暴斃！軍人3天後躺床再也沒醒…室友開門見他「失去意識」嚇傻



News Url: 
https://news.sina.com.tw/article/20220109/40989418.html

News_title: 跨年夜遭男友放鳥…下秒驚收「劈腿激戰片

### 7. 隨機選出50個PCHOME的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 50;
```

In [23]:
pchome_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 20; """
pchome_test_urls = query_from_db(pchome_test_sql)

In [24]:

def pchome_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-2].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%S") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%SZ")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('PChome date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'calss':'article_text'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        a_tags = article_body_tag.find_all('a')
        if content:
            content = re.sub('(\n)+', '\n', content)
            content = re.sub(r'(相關新聞[\s\S]+)', '', content)
            res_dict['news'] = content 
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PChome url: {} did not process properly'.format(url))

    return res_dict

In [25]:
print_util(pchome_test_urls.values, pchome_content_processor)


News Url: 
https://news.pchome.com.tw/internation/cna/20220109/index-16417064377646718011.html

News_title: 承諾恢復文人治國 馬利軍政府再提新時間表
News_category: 國際新聞
News_fb_app_id: 498546706899526
News_fb_page: 1423378217880597
News_keywords: ai,選舉
News_description: 西非經濟共同體（ECOWAS）即將針對馬利政治危機召開一場利害攸關的峰會，馬利軍政府今天搶先一步提出恢復文人治國的時間表。法新社報導，西非經濟共同體各國領袖預料將在峰會上研議對馬利祭出制裁。軍政府…
Published_time: 2022-01-09 05:33:57
Published_date: 2022-01-09
News: 西非經濟共同體（ECOWAS）即將針對馬利政治危機召開一場利害攸關的峰會，馬利軍政府今天搶先一步提出恢復文人治國的時間表。
法新社報導，西非經濟共同體各國領袖預料將在峰會上研議對馬利祭出制裁。
軍政府先前提出5年內轉由文人治國，但這個想法上週遭西非經濟共同體和馬利主要政黨聯盟否決。
西非經濟共同體調停者、奈及利亞前總統強納森（Goodluck Jonathan）上週造訪馬利期間，已要求馬利軍政府修改這項計畫。
國家電視台報導，馬利外交部長迪歐普（Abdoulaye Diop）和政府發言人邁加（Abdoulaye Maiga）今天前往迦納首都阿克拉（Accra），提交一份新提案給西非經濟共同體輪值主席、迦納總統阿庫佛艾杜（Nana Akufo-Addo）。
報導並未具體說明修改後的提案內容。
上校戈伊塔（Assimi Goita）2020年8月發動政變，將民選總統凱塔（Ibrahim Boubacar Keita）趕下台後，國際間對馬利局勢的擔憂與日俱增。
在制裁威脅下，戈伊塔隨後承諾舉行總統和立法選舉，並在2022年2月前恢復文人統治。但他2021年5月策動二次政變，迫使臨時文人政府下台，並宣布自己為臨時總統。
馬利受制於聖戰士叛亂，全國大部分地區都不在政府掌控之中。



News Url: 
https://news.pchome.co

### 8. 隨機選出50個Ettoday的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 50;
```

In [46]:
ettoday_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY created_at DESC
LIMIT 5; """
ettoday_test_urls = query_from_db(ettoday_test_sql)

In [47]:
ettoday_test_urls.iloc[0]['news_url']

'https://www.ettoday.net/news/20220111/2166576.htm?'

In [45]:
url = 'https://www.ettoday.net/news/20220110/2165495.html'
r = requests.get(url, headers = headers)
r.encoding='utf-8'
web_content = r.text
soup = BeautifulSoup(web_content, "lxml")
soup

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/NewsArticle" lang="zh-Hant" prefix="og:http://ogp.me/ns#" xmlns:wb="http://open.weibo.com/wb">
<head>
<meta charset="utf-8"/>
<link href="https://pets.ettoday.net/amp/amp_news.php7?news_id=2165495" rel="amphtml"/>
<title>擠機車歡送媽...屁哈「1掌巴這裡」當墊腳　牠露禮貌笑：你完蛋了 | ETtoday寵物雲 | ETtoday新聞雲</title>
<link href="https://pets.ettoday.net/news/2165495" itemprop="mainEntityOfPage" rel="canonical"/>
<!-- 網頁社群分享meta 開始 -->
<meta content="https://pets.ettoday.net/news/2165495" itemprop="url" property="og:url"/>
<meta content="擠機車歡送媽...屁哈「1掌巴這裡」當墊腳　牠露禮貌笑：你完蛋了 | ETtoday寵物雲 | ETtoday新聞雲" itemprop="name" property="og:title"/>
<meta content="https://cdn2.ettoday.net/images/6117/d6117751.jpg" itemprop="image" property="og:image"/>
<meta content="image/jpeg" property="og:image:type"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="article" property="og:type"/>
<meta content="這一掌來得措手不及啊

In [49]:
def ettoday_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    # Ettoday did not put fb page tag on the content
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    #if fb_page_tag:
        #res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('time', attrs = {'itemprop': 'datePublished'}) 
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get_text().strip(), "%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e2:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get_text().strip(), "%Y年%m月%d日 %H:%M")
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e3:
                print(e3)
                content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))

    

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    temp_content = []
    links, links_descs = [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        
        if p_tags:
            for p in p_tags:
                # Ignore the image caption
                if p.find('strong'):
                    for match in p.find_all('strong', attrs = {'class': 'figcaption'}):
                        match.decompose()
                #    continue
                if p.text:
                    
                    temp_content.append(p.text.strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip().replace('\u3000', ' '))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Ettoday url: {} did not process properly'.format(url))

    return res_dict

In [50]:
print_util(ettoday_test_urls.values, ettoday_content_processor)


News Url: 
https://www.ettoday.net/news/20220111/2166576.htm?

News_title: 大小S選誰？哈林秒回「那邊爛掉也不選妳們」
News_category: ETtoday星光雲
News_fb_app_id: 146858218737386
News_keywords: 小s,徐熙娣,熙娣想聊,哈林
News_description: 女星小S徐熙娣主持的OTT綜藝節目《熙娣想聊》最新一集請到「音樂頑童」哈林（庾澄慶）。兩人進到超市便開始互虧，哈林搞笑吼：「打她的頭！」小S嗆：「你會打老婆？！」他配合回應：「對！」小S更放大絕自曝：「我的天哪！我以為只有我老公會家暴！」並接著虧哈林：「難怪你會結兩次婚。」(小s,徐熙娣,熙娣想聊,哈林)
Published_time: 2022-01-11 04:07:00
Published_date: 2022-01-11
News: 記者許逸群／台北報導
女星小S徐熙娣主持的OTT綜藝節目《熙娣想聊》最新一集請到「音樂頑童」哈林（庾澄慶）。兩人進到超市便開始互虧，哈林搞笑吼：「打她的頭！」小S嗆：「你會打老婆？！」他配合回應：「對！」小S更放大絕自曝：「我的天哪！我以為只有我老公會家暴！」並接著虧哈林：「難怪你會結兩次婚。」
▲小S與哈林一碰面就熱情獻唱。（圖／CATCHPLAY提供）
在本集的「終極二選一單元」中，小S出難題考哈林：「如果可以再讓你結婚一次，你會選擇大S還是小S？」沒想到哈林毫不猶豫地說：「我就算那邊爛掉，我也不選妳們兩個！」讓小S生氣大喊：「為什麼！」誇張的反應讓人捧腹大笑。
▲小S與哈林頻頻互相爆料。（圖／CATCHPLAY提供）
事實上，小S與哈林私下交情不次，小S事前也錄影喊話，表示一直很崇拜他，且常常喝醉時打給對方，「他都非常有耐心聽完我講一堆廢話，隔天還會問他一樣的事，他就會說，我昨天不是才跟你講過了嗎？我就覺得，他真的是一個大好人，好想要跟他見面聊聊喔！」
私下就會常常通電話保持聯絡的小S與哈林，甚至互相爆料自己醫美整形內幕，小S說：「我跟哈林哥真的太熟了，我們在《熙娣想聊》裡的樣子，完全就是我們私下相處的真實模樣，哈林哥會一直鬧我，我也會弄回去，常常被彼此捉弄得哭笑不得！」節目將在本周四播出。



News Url: 
h

### 9. 隨機選出50個Rti 中央廣播電臺的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 50;
```

In [29]:
rti_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 20; """
rti_test_urls = query_from_db(rti_test_sql)

In [30]:
def rti_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    reserved_keywords = ['Rti', '中央廣播電臺', 'Radio Taiwan International']
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ', 1)
        res_dict['news_title'] = title_category[0]
    category_tag = soup.find('div', attrs = {'class':'swiper-wrapper'})
    if category_tag:
        
        res_dict['news_category'] = category_tag.find('a', attrs = {'class':'active'}).get_text().strip()

    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        keyword_lst = [keyword for keyword in keywords_tag['content'].split(',') if keyword not in reserved_keywords]
        res_dict['news_keywords'] = ','.join(keyword_lst)

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if description_tags:
        res_dict['news_description'] = description_tags[1]['content']

    time_tag = soup.find('li', attrs = {'class': 'date'})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M")
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M:%S")
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                #content_parser.logger.info('RTI url: {} date not process properly, error message {}'.format(url, e2))

    article_body_tag = soup.find('article', attrs = {'class' : None})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('RTI url: {} did not process properly'.format(url))
    return res_dict

In [31]:
print_util(rti_test_urls.values, rti_content_processor)



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121652']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121637']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121641']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121622']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121640']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121639']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121628']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121644']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121632']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121624']



list index out of range
ERROR URL:
['https://www.rti.org.tw/news/view/id/2121623']



list index out of range
ERROR URL:
['https://www.rti

### 10. 隨機選出50個公視新聞網的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 50;
```

In [32]:
pts_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 20; """
pts_test_urls = query_from_db(pts_test_sql)

In [33]:
def pts_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit(' | ', 1)
        res_dict['news_title'] = title_category[0].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if len(description_tags) > 1:
        res_dict['news_description'] = description_tags[1]['content']
        
    time_tag_2 = soup.find('div', attrs = {'class': 'maintype-wapper hidden-lg hidden-md'})
    time_tag = soup.find('time', attrs = {'class': None})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get_text(), "%Y-%m-%d %H:%M") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get_text(), "%Y-%m-%d %H:%M:%S") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
            
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        date_tag = time_tag.find('h2')
        if date_tag:
            try:
                d1 = datetime.datetime.strptime(date_tag.text, "%Y年%m月%d日")
                d2 = d1 + datetime.timedelta(hours=8)
                d2 = d2.date()
                #d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e1:
                print(e1)
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e1, url))
    

    article_body_tag = soup.find('div', attrs = {'class':'article_content'})
    article_body_tag_2 = soup.find('article', attrs = {'class': 'post-article'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        if content:
            res_dict['news'] = content
    elif article_body_tag_2:
        content = article_body_tag_2.text.strip()
        if content:
            res_dict['news'] = content
    
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PTS url: {} did not process properly'.format(url))

    return res_dict

In [34]:
print_util(pts_test_urls.values, pts_content_processor)

### 11. 隨機選出50個新頭殼要聞的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 50;
```

In [35]:
newstalk_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 20; """
newstalk_test_urls = query_from_db(newstalk_test_sql)

In [36]:
def newstalk_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d2 = d1.date()
            d1 -= datetime.timedelta(hours=8)
            db_time_format = '%Y-%m-%d %H:%M:%S'
            time_res = d1.strftime(db_time_format)
            res_dict['published_time'] = time_res
            
            db_date_format = '%Y-%m-%d'
            date_res = d2.strftime(db_date_format)
            res_dict['published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d2 = d1.date()
                d1 -= datetime.timedelta(hours=8)
                db_time_format = '%Y-%m-%d %H:%M:%S'
                time_res = d1.strftime(db_time_format)
                res_dict['published_time'] = time_res
                
                db_date_format = '%Y-%m-%d'
                date_res = d2.strftime(db_date_format)
                res_dict['published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('NewsTalk date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    #print(article_body_tag)
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')#, attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('NewsTalk url: {} did not process properly'.format(url))

    return res_dict

In [37]:
print_util(newstalk_test_urls.values, newstalk_content_processor)


News Url: 
https://newtalk.tw/news/view/2022-01-08/693566

News_title: 顏清標將抱病上台為兒催票? 最新醫囑出爐
News_category: 政治
News_fb_app_id: 241741912846250
News_fb_page: 100007902165474
News_keywords: 中二補選,選前之夜,顏清標,顏寬恒
News_description: 台中市第二選區立委補選將在明( 9 )日登場，國民黨候選人顏寬恒的父親顏清標被傳出因身體不適送醫，有靜宜大學的網友在 Dcard 貼出顏清標躺在病床的照片，外傳顏清標將抱病在選前之夜為兒子站台催票，但醫師的最新醫囑也出爐，「今天不能出院」。據傳顏清標在 6 日下午4時，因頭暈、身體不適到院急診，院方
Published_time: 2022-01-08 07:14:19
Published_date: 2022-01-08
News: 台中市第二選區立委補選將在明( 9 )日登場，國民黨候選人顏寬恒的父親顏清標被傳出因身體不適送醫，有靜宜大學的網友在 Dcard 貼出顏清標躺在病床的照片，外傳顏清標將抱病在選前之夜為兒子站台催票，但醫師的最新醫囑也出爐，「今天不能出院」。
據傳顏清標在 6 日下午4時，因頭暈、身體不適到院急診，院方安排一系列檢查、檢驗，發現顏清標疑似出現脫水，血壓不穩定，因此安排住院治療。不過有網友質疑，顏清標躺在醫院病床上，卻沒有吊點滴。
病患出院需要徵得醫師評估病況後決定，但病患如果堅持出院仍可以出院。據了解，只要家屬或病患本人簽署切結書仍可出院。但依照目前醫囑，「顏清標今天無法出院」。
顏寬恒今日受訪時表示，「父親長期勞累，所以身體不舒服，當下就在旁人攙扶下到醫院掛急診，目前已有比較穩定了，謝謝大家關心」。
面對媒體詢問顏清標是否因為「被對手質疑祖墳問題」，被氣到身體出狀況，顏寬恒不禁哽咽、眼眶略紅的說，「謝謝大家關心，暫不談此事，只希望爸爸身體能趕快恢復」。



News Url: 
https://newtalk.tw/news/view/2021-12-08/678306

News_title: 跟著法人衝一波！籌碼達人破投信布局眉角 12檔作帳標的出列 最看好「這族群」
News_categ

In [38]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import jieba
import numpy as np
# 如果檔案內有一些編碼錯誤，使用 errors='ignore' 來忽略錯誤
with open("107.csv", encoding="big5", errors='ignore') as f:
    text = f.read()

# 設定使用 big5 斷詞
jieba.set_dictionary('dict.txt.big')
wordlist = jieba.cut(text)
words = " ".join(wordlist)
#文字雲造型圖片
mask = np.array(Image.open('picture.png')) #文字雲形狀
# 從 Google 下載的中文字型
font = 'SourceHanSansTW-Regular.otf'
#背景顏色預設黑色，改為白色、使用指定圖形、使用指定字體
my_wordcloud = WordCloud(background_color='white',mask=mask,font_path=font).generate(words)

plt.imshow(my_wordcloud)
plt.axis("off")
plt.show()
#存檔
wordcloud.to_file('word_cloud.png')

ModuleNotFoundError: No module named 'matplotlib'

In [33]:
import numpy as np
np.log(100)


4.605170185988092

In [35]:
import sys
sys.version

'3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'